In [1]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import InMemorySaver
import uuid
import time

In [2]:
checkpointer = InMemorySaver()

@task
def dummy_task(topic:str) -> str:
    time.sleep(1)
    return f"dummy task: {topic}"

@entrypoint(checkpointer=checkpointer)
def workflow(topic: str) -> str:
    result = dummy_task(topic).result()
    return f"workflow result: {result}"

thread_id = str(uuid.uuid4())

config = {
    "configurable":{
        "thread_id": thread_id
    }
}

for i in workflow.stream("hello", config=config):
    print(i)

{'dummy_task': 'dummy task: hello'}
{'workflow': 'workflow result: dummy task: hello'}


In [3]:
# parallel execution 
@task
def num_pow__and_add_one(num: int) -> int:
    return (num * num) + 1

@entrypoint(checkpointer=checkpointer)
def workflow(num_list: list[int])->list[int]:
    return [num_pow__and_add_one(x).result() for x in num_list]

thread_id = str(uuid.uuid4())

config = {
    "configurable":{
        "thread_id": thread_id
    }
}

# for i in workflow.stream([1,3,4, 528], config=config):
#     print(i)

result = workflow.invoke([1,3,4,65], config=config)
print(result)

[2, 10, 17, 4226]


In [4]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
import operator

class State(TypedDict):
    messages: Annotated[str, operator.add]
    ans: str

def node_1(state: State):
    return {"messages": f"node: 1 append: {state["messages"]}, "}

def node_2(state: State)-> str:
    return {"messages": f"node: 2 append: {state["messages"]}"}

builder = StateGraph(State).add_node("node_1", node_1).add_node("node_2", node_2)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", END)
graph = builder.compile()


In [5]:
@task
def call_graph(text):
    result = graph.invoke({"messages": text})
    return result

@entrypoint(checkpointer=checkpointer)
def workflow(text):
    result = call_graph(text).result()
    return result

workflow.invoke("spce", config=config)

{'messages': 'spcenode: 1 append: spce, node: 2 append: spcenode: 1 append: spce, '}

In [6]:
for i in workflow.stream("space", stream_mode=["updates"], config=config):
    print(i)

('updates', {'call_graph': {'messages': 'spacenode: 1 append: space, node: 2 append: spacenode: 1 append: space, '}})
('updates', {'workflow': {'messages': 'spacenode: 1 append: space, node: 2 append: spacenode: 1 append: space, '}})


In [11]:
# retry policy
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import RetryPolicy, CachePolicy

checkpointer = MemorySaver()

count = 0

@task(retry_policy=RetryPolicy(retry_on=ValueError), cache_policy=CachePolicy(ttl=15))
def dummy_task(text:str):
    global count
    count += 1

    if count > 2:
        raise ValueError
    return f"count: {count}, text: {text}"

@entrypoint(checkpointer=checkpointer)
def workflow(num: int) -> str:
    result = [dummy_task("text").result() for x in range(num)]
    return result

thread_id = str(uuid.uuid4())

config = {
    "configurable":{
        "thread_id": thread_id
    }
}

result = workflow.invoke(2, config=config)
print(result)

['count: 1, text: text', 'count: 2, text: text']


In [ ]:
# human-in-loop, use Command and Interrupt
from langgraph.func import entrypoint, task
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
import uuid

checkpointer = MemorySaver()

@task
def step_1(text:str) -> str:
    return f"Step:1 Complete, text: {text}"

@task
def human_intervation(text: str) -> str:
    user_input = interrupt("Enter random value")
    return f"user enter value: {user_input}-{text}"

@task
def step_2(text: str) -> str:
    return f"Step:2 Complete, text: {text}"

@entrypoint(checkpointer=checkpointer)
def workflow(text: str) -> str:
    first_step = step_1(text).result()
    human_step = human_intervation(text).result()
    second_step = step_2(text).result()
    return first_step and human_step and second_step

thread_id = str(uuid.uuid4())

config = {
    "configurable":{
        "thread_id": thread_id
    }
}

for i in workflow.stream("space", config=config):
    print(i)

{'step_1': 'Step:1 Complete, text: space'}
{'__interrupt__': (Interrupt(value='Enter random value', resumable=True, ns=['workflow:88d51e27-6a0c-24a2-0515-e709e2d8c360', 'human_intervation:aed19737-fe0b-88a9-4bc8-fea81b2a766a']),)}


In [14]:
for i in workflow.stream(Command(resume="sample"), config=config):
    print(i)

{'human_intervation': 'user enter value: sample-space'}
{'step_2': 'Step:2 Complete, text: space'}
{'workflow': 'Step:2 Complete, text: space'}


In [15]:
list(workflow.get_state_history(config))

[StateSnapshot(values='Step:2 Complete, text: space', next=(), config={'configurable': {'thread_id': 'efd4bfc7-cfd6-4508-82d9-07e3314c6acd', 'checkpoint_ns': '', 'checkpoint_id': '1f06e971-f6a3-61b2-8000-d3ebbee8f206'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-01T05:19:39.465556+00:00', parent_config={'configurable': {'thread_id': 'efd4bfc7-cfd6-4508-82d9-07e3314c6acd', 'checkpoint_ns': '', 'checkpoint_id': '1f06e971-e59d-6b2d-bfff-145fd8f79527'}}, tasks=(), interrupts=()),
 StateSnapshot(values=None, next=('workflow',), config={'configurable': {'thread_id': 'efd4bfc7-cfd6-4508-82d9-07e3314c6acd', 'checkpoint_ns': '', 'checkpoint_id': '1f06e971-e59d-6b2d-bfff-145fd8f79527'}}, metadata={'source': 'input', 'step': -1, 'parents': {}}, created_at='2025-08-01T05:19:37.680768+00:00', parent_config=None, tasks=(PregelTask(id='88d51e27-6a0c-24a2-0515-e709e2d8c360', name='workflow', path=('__pregel_pull', 'workflow'), error=None, interrupts=(Interrupt(value='E